In [10]:
from flask import Flask ,render_template,request

app=Flask(__name__)
@app.route('/')
def index():
    return render_template('index.html')



In [11]:
if __name__ == '__main__':
    app.run(debug=True,port=4000,use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:4000
Press CTRL+C to quit
127.0.0.1 - - [13/Oct/2024 03:04:49] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Oct/2024 03:04:49] "GET /static/assets/css/style.css HTTP/1.1" 404 -
127.0.0.1 - - [13/Oct/2024 03:04:49] "GET /static/assets/css/responsive.css HTTP/1.1" 404 -
127.0.0.1 - - [13/Oct/2024 03:04:49] "GET /static/assets/images/blog/sidebar-06.jpg HTTP/1.1" 404 -
127.0.0.1 - - [13/Oct/2024 03:04:49] "GET /static/assets/images/avatar/avatar-small-09.png HTTP/1.1" 404 -
127.0.0.1 - - [13/Oct/2024 03:04:49] "GET /static/assets/images/logo/logo.png HTTP/1.1" 404 -
127.0.0.1 - - [13/Oct/2024 03:04:49] "GET /static/assets/images/box-icon/upload.png HTTP/1.1" 404 -
127.0.0.1 - - [13/Oct/2024 03:04:49] "GET /static/assets/images/box-icon/coin-01.png HTTP/1.1" 404 -
127.0.0.1 - - [13/Oct/2024 03:04:49] "GET /static/assets/images/box-icon/coin-02.png HTTP/1.1" 404 -
127.0.0.1 - - [13/Oct/2024 03:04:49] "GET /static/assets/images/box-icon/coin-03.png HTTP/1.1" 404 -
